# Howso Engine Inference 

## Overview 

 
Howso Engine is a generalized Machine Learning (ML) and Artificial Intelligence platform that creates powerful decision-making models that are fully explainable, auditable, and editable. Howso Engine uses Instance-Based Machine Learning which stores instances, i.e., data points, in memory and makes predictions about new instances given their relationship to existing instances. This technology harnesses a fast spatial query system and techniques from information theory to enhance both performance and accuracy. 


In this notebook we will explore the most basic workflow for training Howso Engine and making predictions. 


In [1]:
from pmlb import fetch_data

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes

# Step 1: Import Data

Our example dataset for this recipe is the well known `Adult` dataset. This dataset consists of 14 Context Features and 1 Action Feature. The Action Feature in this version of the `Adult` dataset has been renamed to `target` and it takes the form of a binary indicator for whether a person in the data makes more than $50,000/year (*target*=1) or less (*target*=0).

### **Definitions:**

**`Action Feature`:** A feature that the user would like to discriminatively predict or generatively synthesize. In traditional ML these may be referred to as targets, target features, or dependent variables.

**`Context Feature`:** A feature that is used as an input to the model that creates a context from which a prediction is derived. In traditional ML these are often referred to as features, inputs, predictors, or independent features.

In [2]:
df = fetch_data('adult', local_cache_dir="../../data/adult")

# Subsample the data to ensure the example runs quickly
df = df.sample(2001)

# Split out the last row for a test set
df_test = df.iloc[[-1]].copy()
df_test = df_test.drop('target', axis=1)

df.drop(df.index[-1], inplace=True)

# Step 2: Feature Mapping

In the Howso Engine workflow, feature attributes are an essential part of model building and usage. By incorporating certain feature attributes into the model itself, Howso Engine gains another layer of information that will help in fine-tuning the results. 

In order to assist the user with defining the feature attributes, Howso provides the `infer_feature_attributes` function that automatically processes the dataset for the user.

These feature attributes are based on the existing data, rather than exact descriptive statistics. This is why, for example, the min and max bounds on continuous features are not the exact min and max values of the dataset, but rather an expanded version of those min and max values to allow for some variation. However, users are encouraged to manually adjust feature attributes such as bounds if the properties of any features in the data are well defined.

Additionally, Howso always encourages users to inspect the feature attributes of their data. Incorrect feature attributes can severely impact the accuracy of the model. Specific errors to lookout for are incorrect values for 'type' or 'bounds'.

In [3]:
# Infer features attributes
features = infer_feature_attributes(df)
features

{'age': {'type': 'continuous',
  'decimal_places': 0,
  'original_type': {'data_type': 'numeric', 'size': 8},
  'bounds': {'min': 7.0, 'max': 148.0}},
 'workclass': {'type': 'nominal',
  'data_type': 'number',
  'decimal_places': 0,
  'original_type': {'data_type': 'integer', 'size': 8},
  'bounds': {'allow_null': False}},
 'fnlwgt': {'type': 'continuous',
  'decimal_places': 0,
  'original_type': {'data_type': 'numeric', 'size': 8},
  'bounds': {'min': 8103.0, 'max': 1202604.0}},
 'education': {'type': 'nominal',
  'data_type': 'number',
  'decimal_places': 0,
  'original_type': {'data_type': 'integer', 'size': 8},
  'bounds': {'allow_null': False}},
 'education-num': {'type': 'continuous',
  'decimal_places': 0,
  'original_type': {'data_type': 'numeric', 'size': 8},
  'bounds': {'min': 1.0, 'max': 20.0}},
 'marital-status': {'type': 'nominal',
  'data_type': 'number',
  'decimal_places': 0,
  'original_type': {'data_type': 'integer', 'size': 8},
  'bounds': {'allow_null': False}},
 

To prepare to measure model performance, we also define our action features and context features that we want to use in our experiment.

In [4]:
# Specify Context and Action Features
action_features = ['target']
context_features = features.get_names(without=action_features)

# Step 3: Create Trainee

To begin the Howso Engine workflow, a Trainee is created to act as a base for all of our ML needs. In all subsequent notebooks, we will refer to Howso Engine's model as the Trainee.

### **Definitions:**

**`Trainee`:** A collection of Cases that comprise knowledge. It also includes metadata and parameters. In traditional ML this is referred to as a model.

**`Case`:** A set of feature values representing an observation. In traditional ML, a Case is sometimes referred to as an "observation", "record", or "data point". In database terms, a Case would be a row of values. For supervised learning a Case is a set of Context values and Action values. For unsupervised learning a Case is just a set of feature values. 

In [5]:
# Create the Trainee
t = Trainee(
    features=features,
    overwrite_existing=True
)

Version 18.1.2 of Howso Engine™ is available. You are using version 18.0.2.


# Step 4: Preprocessing and Training

One benefit of Howso Engine is that most standard forms of data pre-processing such as one-hot encoding and standardization are **NOT** needed, which is in contrast to many modern ML models. This does not include more sophisticated forms of pre-processing such as feature selection or feature engineering, which may still be useful. Fitting is also done in two steps in Howso Engine.

### **Definitions:**

**`Train`:** To add a case or set of cases to the Trainee, making the information contained in these cases available for downstream tasks.

**`Analyze`:** Tune internal parameters to improve performance and accuracy of predictions and metrics. Analysis may be targeted or targetless.  Targetless analysis provides the best balanced set of parameters if an Action Feature is not specified, along with a performance boost while targeted analysis provides a boost to accuracy towards the specified action features. 


In [6]:
# Train
t.train(df)

# Analyze the Trainee
# (By specifying action_features, this becomes a Targeted analysis)
t.analyze(context_features=context_features, action_features=action_features)

# Step 5: Inference

Once Howso Engine is trained and analyzed, it provides the user with a variety of ML capabilities. One of the core capabilities in engine is prediction, which Engine infers a action value based on the given context values. This prediction is a standard output when using `react` and the prediction can be found by navigating the output as shown.


In [12]:
results = t.react(
    df_test,
    context_features=context_features,
    action_features=action_features,
)

predictions = results['action'][action_features]
predictions

,target
0,1


### Predict Function

A `predict` method is also provided as an ease-of-use function that provides the same results as the method shown above using `react` and directly returns the predictions without requiring the user to navigate the output.

In [10]:
predictions = t.predict(
    df_test,
    context_features=context_features,
    action_features=action_features,
)

predictions